# 텍스트의 분포로 벡터화 하기

## 단어 빈도를 이용한 벡터화 Bag of Words

### keras Tokenizer 활용

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence) # 단어장 생성
bow = dict(tokenizer.word_counts) # 각 단어와 각 단어의 빈도를 bow에 저장

print("Bag of Words :", bow) # bow 출력
print('단어장(Vocabulary)의 크기 :', len(tokenizer.word_counts)) # 중복을 제거한 단어들의 개수

Bag of Words : {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(Vocabulary)의 크기 : 10


### scikit-learn CountVectorizer 활용

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

vector = CountVectorizer()
bow = vector.fit_transform(sentence).toarray()

print('Bag of Words : ', bow) # 코퍼스로부터 각 단어의 빈도수를 기록한다.
print('각 단어의 인덱스 :', vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

Bag of Words :  [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스 : {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


In [3]:
print('단어장(Vocabulary)의 크기 :', len(vector.vocabulary_))

단어장(Vocabulary)의 크기 : 10


- Keras Tokenizer는 딥러닝 모델의 임베딩 레이어와 함께 사용될 때 강력하며, 텍스트를 정수 시퀀스로 변환하는 데 중점을 둔다.  
토큰화와 인덱싱에 초점을 맞추고, 그 외의 전처리는 비교적 간단하다.  
- scikit-learn CountVectorizer는 BoW 표현 자체를 생성하고, 더 많은 전처리 및 특성 추출 옵션을 내장하고 있어 다양한 전통적인 머신러닝 모델에 바로 적용하기 편리하다.  
출력 형태도 희소 행렬로 효율적이다.

## DTM과 코사인 유사도

문서1 : I like dog  
문서2 : I like cat  
문서3 : I like cat I like cat  

In [4]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

doc1 = np.array([0,1,1,1]) # 문서1 벡터
doc2 = np.array([1,0,1,1]) # 문서2 벡터
doc3 = np.array([2,0,2,2]) # 문서3 벡터

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [5]:
print('{:.2f}'.format(cos_sim(doc1, doc2))) #문서1과 문서2의 코사인 유사도
print('{:.2f}'.format(cos_sim(doc1, doc3))) #문서1과 문서3의 코사인 유사도
print('{:.2f}'.format(cos_sim(doc2, doc3))) #문서2과 문서3의 코사인 유사도

0.67
0.67
1.00


DTM에서 코사인 유사도는 0 이상 1 이하의 값을 가지며 값이 1에 가까울수록 유사도가 높다고 판단

### scikit-learn CountVectorizer 활용

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'John likes to watch movies',
    'Mary likes movies too',
    'Mary also likes to watch football games',
]
vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도수를 기록.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


## TF-IDF

모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며,  
특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단하는 것

In [59]:
from math import log
import pandas as pd

In [60]:
docs = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',
]

In [9]:
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
print('단어장의 크기 :', len(vocab))
print(vocab)

단어장의 크기 : 13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


In [10]:
N = len(docs) # 총 문서의 수
N

3

In [62]:
def tf(t, d): # 단어 빈도
    return d.count(t)

def idf(t): # 역문서 빈도
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1)) + 1 

def tfidf(t, d): # 최종 TF-IDF
    return tf(t,d)* idf(t)

In [63]:
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab) # 각 문서 내에서 단어의 빈도수 계산하여 DTM 형태 생성
tf_

,also,and,football,games,james,john,likes,mary,movies,to,too,tv,watch
0,0,1,0,0,0,0,2,0,2,2,1,0,1
1,0,0,0,0,0,0,1,0,0,1,0,0,1
2,1,0,1,1,0,0,1,0,0,1,0,0,1


In [64]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns=["IDF"]) # 각 단어의 희귀도를 나타내는 IDF 계산 
idf_

,IDF
also,1.405465
and,1.405465
football,1.405465
games,1.405465
james,2.098612
john,2.098612
likes,0.712318
mary,2.098612
movies,1.405465
to,0.712318


In [14]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,1.0,0.000000,0.000000,1.405465,0.000000,0.000000,1.424636,2.81093,1.424636,1.405465,0.712318
1,1.405465,0.000000,0.0,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,0.00000,0.712318,0.000000,0.712318
2,0.000000,0.000000,1.0,0.000000,1.405465,0.000000,1.405465,1.405465,0.712318,0.00000,0.712318,0.000000,0.712318


### scikit-learn TFidVectorizer 활용

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',
]

tfidfv = TfidfVectorizer().fit(corpus)
vocab = list(tfidfv.vocabulary_.keys()) # 단어장을 리스트로 저장
vocab.sort() # 단어장을 알파벳 순으로 정렬

# TF-IDF 행렬에 단어장을 데이터프레임의 열로 지정하여 데이터프레임 생성
tfidf_ = pd.DataFrame(tfidfv.transform(corpus).toarray(), columns = vocab)
tfidf_

,also,and,football,games,james,john,likes,mary,movies,to,too,tv,watch
0,0.000000,0.321556,0.000000,0.000000,0.000000,0.321556,0.379832,0.244551,0.643111,0.189916,0.321556,0.000000,0.189916
1,0.000000,0.000000,0.000000,0.000000,0.572929,0.000000,0.338381,0.000000,0.000000,0.338381,0.000000,0.572929,0.338381
2,0.464997,0.000000,0.464997,0.464997,0.000000,0.000000,0.274634,0.353642,0.000000,0.274634,0.000000,0.000000,0.274634


## LSA와 LDA

### LSA
단어와 단어 사이, 문서와 문서 사이, 단어와 문서 사이의 의미적 유사성 점수를 찾아낼 수 있다.
- 의미적 유사성 파악 (DTM/TF-IDF 한계 극복)
- 특잇값 분해 -> 핵심 원리
- Truncated SVD (절단된 특잇값 분해)의 활용 -> 차원 축소 및 노이즈 제거


LSA는 DTM/TF-IDF 행렬에 Truncated SVD를 적용하여 텍스트의 표면적인 단어 빈도 관계를 넘어선 잠재적인 의미 구조나 주제를 발견하고, 이를 통해 문서나 단어를 더 풍부한 의미 벡터로 표현하는 강력한 방법이다.


In [77]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [17]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# 이거 추가로 다운로드해야함
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juwonhee\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\juwonhee\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juwonhee\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\juwonhee\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [65]:
import os

csv_filename = './abcnews-date-text.csv'

# 파일 다운로드
url = "https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv"
urllib.request.urlretrieve(url, filename=csv_filename)

('./abcnews-date-text.csv', <http.client.HTTPMessage at 0x24dede07710>)

In [19]:
data = pd.read_csv(csv_filename, on_bad_lines='skip')
data.shape

(1082168, 2)

In [20]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [66]:
text = data[['headline_text']].copy() # 헤드라인만 별도로 저장
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [22]:
text.nunique() # 중복을 제외하고 유일한 시퀀스를 가지는 샘플의 개수를 출력

headline_text    1054983
dtype: int64

In [23]:
text.drop_duplicates(inplace=True) # 중복 샘플 제거
text.reset_index(drop=True, inplace=True)
text.shape

(1054983, 1)

중복 제거를 통해, 고유한 헤드라인 텍스트만을 선별하여 LSA 분석의 품질과 효율성을 높이는 중요한 전처리 단계

### 데이터 정제 및 정규화

In [69]:
import nltk # 불용어 제거


In [70]:
# NLTK 토크나이저를 이용해서 토큰화
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

# 불용어 제거
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


In [71]:
# 단어 정규화. 3인칭 단수 표현 -> 1인칭 변환, 과거형 동사 -> 현재형 동사 등을 수행한다.
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1 ~ 2인 단어는 제거.
text = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 2])
print(text[:5])

0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object


### 역토큰화 및 DTM 생성

In [72]:
# 역토큰화 (토큰화 작업을 역으로 수행)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(text[i])
    detokenized_doc.append(t)

train_data = detokenized_doc

In [73]:
train_data[:5]

['aba decide community broadcast licence',
 'act fire witness must aware defamation',
 'call infrastructure protection summit',
 'air staff aust strike pay rise',
 'air strike affect australian travellers']

In [74]:
# 상위 5000개의 단어만 사용
c_vectorizer = CountVectorizer(stop_words='english', max_features = 5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)

어떤 단어들을 사용할지, 그리고 그 단어들에 어떤 번호를 부여할지를 배운 다음(fit),  
이 정보를 바탕으로 train_data를 실제 숫자 행렬(DTM)로 바꿔주는(transform) 역할을 하는 것

In [75]:
print('행렬의 크기 :',document_term_matrix.shape)

행렬의 크기 : (1082168, 5000)


### scikit-learn TruncatedSVD 활용

In [76]:
from sklearn.decomposition import TruncatedSVD

n_topics = 10
lsa_model = TruncatedSVD(n_components = n_topics)
lsa_model.fit_transform(document_term_matrix)

array([[ 1.13452631e-02, -2.06117113e-03,  1.88100974e-02, ...,
         3.58555830e-03,  3.33208297e-03,  1.50474264e-02],
       [ 2.87876421e-02, -1.02560282e-02,  1.97318629e-02, ...,
        -2.72921967e-03, -1.02282521e-02, -8.67213609e-03],
       [ 4.69602949e-03, -1.07481314e-03,  1.00409326e-02, ...,
        -2.79001771e-03,  1.89889871e-03,  2.30803708e-03],
       ...,
       [ 2.88650348e-02,  6.58098851e-03,  2.61430500e-02, ...,
         3.39556944e-02,  3.87246578e-03,  2.52481602e-02],
       [ 5.72967081e-02,  1.08118295e-02,  1.42286510e-01, ...,
         8.50643857e-01,  6.86302343e-01, -3.83113172e-01],
       [ 7.11383380e-02,  2.59634045e-02, -9.28163276e-04, ...,
         1.30104944e-03,  2.47304993e-04, -3.85534394e-02]])

In [32]:
print(lsa_model.components_.shape)

(10, 5000)


In [33]:
terms = c_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lsa_model.components_, terms)

Topic 1: [('police', 0.74693), ('man', 0.45386), ('charge', 0.21101), ('new', 0.14082), ('court', 0.11142)]
Topic 2: [('man', 0.69459), ('charge', 0.30008), ('court', 0.16769), ('face', 0.11445), ('murder', 0.10626)]
Topic 3: [('new', 0.83802), ('plan', 0.23643), ('say', 0.18238), ('govt', 0.1107), ('council', 0.10936)]
Topic 4: [('say', 0.73902), ('plan', 0.36006), ('govt', 0.16799), ('council', 0.12815), ('urge', 0.07737)]
Topic 5: [('plan', 0.7334), ('council', 0.18062), ('govt', 0.14135), ('urge', 0.08207), ('water', 0.066)]
Topic 6: [('govt', 0.54532), ('court', 0.25086), ('urge', 0.24024), ('fund', 0.21669), ('face', 0.16267)]
Topic 7: [('charge', 0.49764), ('court', 0.4568), ('face', 0.39123), ('plan', 0.11388), ('murder', 0.10506)]
Topic 8: [('win', 0.60052), ('court', 0.35553), ('kill', 0.19073), ('crash', 0.14536), ('australia', 0.09138)]
Topic 9: [('win', 0.59152), ('charge', 0.48745), ('australia', 0.11613), ('cup', 0.07512), ('change', 0.07194)]
Topic 10: [('council', 0.78

## LDA

### TF-IDF 행렬 생성

In [34]:
# 상위 5,000개의 단어만 사용
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

# TF-IDF 행렬의 크기를 확인해봅시다.
print('행렬의 크기 :', tf_idf_matrix.shape)

행렬의 크기 : (1054983, 5000)


### scikit-learn LDA Model 활용

In [35]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_model.fit_transform(tf_idf_matrix)

array([[0.0335099 , 0.0335099 , 0.0335099 , ..., 0.1702395 , 0.0335099 ,
        0.03351907],
       [0.03365628, 0.03365628, 0.03365628, ..., 0.03365628, 0.03365628,
        0.03365628],
       [0.25184095, 0.0366096 , 0.0366096 , ..., 0.45528225, 0.0366096 ,
        0.0366096 ],
       ...,
       [0.26688724, 0.02914502, 0.14077174, ..., 0.02914502, 0.02914502,
        0.02914502],
       [0.02637829, 0.0263834 , 0.11651895, ..., 0.39092402, 0.02637829,
        0.02637829],
       [0.03376055, 0.03376055, 0.03376055, ..., 0.40866263, 0.21169867,
        0.03376966]])

In [36]:
print(lda_model.components_.shape)

(10, 5000)


In [78]:
# LDA의 결과 토픽과 각 단어의 비중을 출력
terms = tfidf_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n-1:-1]])

get_topics(lda_model.components_, terms)


Topic 1: [('home', 4048.78778), ('hit', 3572.60152), ('market', 3141.55263), ('ban', 2995.63559), ('rise', 2909.02651)]
Topic 2: [('australia', 6721.04185), ('perth', 4551.37242), ('kill', 3977.09633), ('year', 3925.53959), ('say', 3395.39353)]
Topic 3: [('say', 7349.92516), ('court', 5250.21134), ('open', 3770.65617), ('state', 3656.60868), ('face', 3611.67807)]
Topic 4: [('man', 6521.51056), ('police', 6333.33358), ('charge', 5948.07591), ('queensland', 5552.61778), ('murder', 4677.1475)]
Topic 5: [('melbourne', 5298.43132), ('school', 3966.56666), ('report', 3792.92855), ('rural', 3521.89517), ('warn', 3379.34835)]
Topic 6: [('australian', 7674.39181), ('world', 4536.14226), ('country', 4168.83782), ('day', 3852.20674), ('crash', 3794.11334)]
Topic 7: [('election', 5416.23599), ('adelaide', 4868.15748), ('south', 4852.30258), ('house', 4481.58381), ('make', 3773.26989)]
Topic 8: [('canberra', 4323.53542), ('2016', 3962.69218), ('win', 3874.8274), ('qld', 3229.28135), ('price', 2762.

## 형태소 분석기와 단어 미동록 문제

In [38]:
en_text = "The dog ran back to the corner near the spare bedrooms"
print(en_text.split())

['The', 'dog', 'ran', 'back', 'to', 'the', 'corner', 'near', 'the', 'spare', 'bedrooms']


In [39]:
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사 왔어"
print(kor_text.split())

['사과의', '놀라운', '효능이라는', '글을', '봤어.', '그래서', '오늘', '사과를', '먹으려고', '했는데', '사과가', '썩어서', '슈퍼에', '가서', '사과랑', '오렌지', '사', '왔어']


In [79]:
!pip install konlpy

In [41]:
from konlpy.tag import Okt

tokenizer = Okt()
print(tokenizer.morphs(kor_text))

['사과', '의', '놀라운', '효능', '이라는', '글', '을', '봤어', '.', '그래서', '오늘', '사과', '를', '먹으려고', '했는데', '사과', '가', '썩어서', '슈퍼', '에', '가서', '사과', '랑', '오렌지', '사', '왔어']


In [42]:
print(tokenizer.morphs('모두의연구소에서 자연어 처리를 공부하는 건 정말 즐거워'))

['모두', '의', '연구소', '에서', '자연어', '처리', '를', '공부', '하는', '건', '정말', '즐거워']


## soynlp

In [81]:
import urllib.request

txt_filename = './2016-10-20.txt'

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt",\
                            filename=txt_filename)

('./2016-10-20.txt', <http.client.HTTPMessage at 0x24df848bf80>)

In [80]:
!pip install soynlp

In [45]:
from soynlp import DoublespaceLineCorpus

# 말뭉치에 대해서 다수의 문서로 분리
corpus = DoublespaceLineCorpus(txt_filename)
len(corpus)

30091

In [46]:
i = 0
for document in corpus:
  if len(document) > 0:
    print(document)
    i = i+1
  if i == 3:
    break

19  1990  52 1 22
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에

### 응집 확률과 브랜칭 엔트로피 단어 점수표

In [47]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 1.832 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


### 응집 확률

In [48]:
word_score_table["반포한"].cohesion_forward

0.08838002913645132

In [49]:
word_score_table["반포한강"].cohesion_forward

0.19841268168224552

In [50]:
word_score_table["반포한강공"].cohesion_forward

0.2972877884078849

In [51]:
word_score_table["반포한강공원"].cohesion_forward

0.37891487632839754

In [52]:
word_score_table["반포한강공원에"].cohesion_forward

0.33492963377557666

### 브랜칭 엔트로피

In [53]:
word_score_table["디스"].right_branching_entropy

1.6371694761537934

In [54]:
word_score_table["디스플"].right_branching_entropy

-0.0

In [55]:
word_score_table["디스플레"].right_branching_entropy

-0.0

In [56]:
word_score_table["디스플레이"].right_branching_entropy

3.1400392861792916

### LTokenizer

In [57]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

### 최대 점수 토크나이저

In [58]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']